In [1]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 M

In [1]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
from guidance import gen
from evaluate import load
import csv
import nltk.translate.bleu_score as bleu
from rouge import Rouge
from transformers import BertTokenizer, BertModel
import torch
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import warnings
from typing import Any, Dict, Tuple
import re

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Dataset

- id: unique identifier of the example
- source: original model input for paraphrase generation
- hyp1: first alternative paraphrase of the source
- hyp2: second alternative paraphrase of the source
- label: hyp1 or hyp2, based on which of those has been annotated as hallucination
- type: hallucination category assigned.
  - Possible values: addition, named-entity, number, conversion, date, tense, negation, gender, pronoun, antonym, natural

## Trial dataset

This is a small list of examples, provided to help the participants get familiar with the task. Each example contains the following fields:
- id
- source
- hyp1
- hyp2
- label
- type

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
from datasets import load_dataset

#load the trial data for both English and Swedish
trial_ds = load_dataset("Eloquent/HalluciGen-PG", name="trial")

#load the trial data only for Swedish
trial_ds_sv = load_dataset("Eloquent/HalluciGen-PG", name="trial", split="trial_swedish")

print(trial_ds.keys())

trial_df_en = trial_ds['trial_english'].to_pandas()
trial_df_se= trial_ds['trial_swedish'].to_pandas()

print('English:')

display(trial_df_en.head())

print('Swedish')
display(trial_df_se.head())

dict_keys(['trial_swedish', 'trial_english'])
English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


Swedish


,id,source,type,hyp1,hyp2,label
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1


In [4]:
print(trial_df_en['type'].unique())


['antonym' 'negation' 'named entity' 'natural' 'addition' 'gender'
 'number' 'pronoun' 'conversion' 'date']


In [5]:
print('English:')

display(trial_df_en.head())

English:


,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2


In [6]:
print('English')
row = trial_df_en.iloc[6]  # Selecting the first row
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_en.iloc[5]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_en.iloc[4]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

English
Source: Madam President, I am speaking on behalf of our colleague, Mr Francis Decourrière, who drafted one of the motions for a resolution.
hyp1: One of the motions for a resolution was drafted by Mr Francis Decourrière.
hyp2: One of the motions for a resolution was drafted by Mrs Francis Decourrière.
label: hyp2
type: gender
Source: This state of affairs has not changed in more than 100 years, but hopefully at some stage - and perhaps soon - change will come.
hyp1: There has been no change in the status quo in over 100 years, but there is hope that change will soon come. 
hyp2: The state of affairs is1-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-65561-6556
label: hyp2
type: addition
Source: We need quite specific legisl

In [7]:
print('Swedish')
row = trial_df_se.iloc[0]  # Selecting the first row
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_se.iloc[1]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

row = trial_df_se.iloc[2]
print("Source:", row['source'])
print("hyp1:", row['hyp1'])
print("hyp2:", row['hyp2'])
print("label:",row['label'] )
print("type:",row['type'] )

Swedish
Source: Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
hyp1: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2: Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,6 miljoner dollar.
label: hyp2
type: number
Source: Hongkong-universitet samarbetar med universitet, företag och statliga sektorer i Kina för att samordna utbildningsprogram och forskningscentra för att främja högteknologisk forskning, kommersialisering och tekniköverföring.
hyp1: University of Hong Kong samarbetar med olika kinesiska universitet, företag och regeringsdepartement för att samordna utbildningsprogram och forskningscentra med syftet att främja avancerad teknologisk forskning, kommersialisering och tekniköverföring.
h

In [8]:
#load the test data for the generation step in both English and Swedish
data = load_dataset("Eloquent/HalluciGen-PG", "test_detection")

print(data.keys())

data_df_en = data['test_detection_english'].to_pandas()
data_df_se= data['test_detection_swedish'].to_pandas()


dict_keys(['test_detection_swedish', 'test_detection_english'])


In [9]:
data_df_en

,id,source,hyp1,hyp2
0,0,It has enabled us to support and encourage an ...,You can support and encourage an exchange of e...,We can support and encourage an exchange of ex...
1,1,"Therefore, I am calling for an increase in the...",I want to see a decrease in the premiums for a...,I want to see an increase in the premiums for ...
2,2,"In other words, a person may be prevented from...",When there is reason to fear that a violent ac...,When there is reason to fear that a violent ac...
3,3,None of my 34 amendments were adopted and my a...,Our arguments against the ridiculous administr...,My arguments against the ridiculous administra...
4,4,The final key feature of the report is its ori...,The final main feature of the report is its or...,The final features of the report are its orien...
...,...,...,...,...
114,114,I am referring to the current vote but in rela...,The vote on Amendment No 98 will take place be...,The vote on Amendment No 98 will take place af...
115,115,When I became a Member of the European Parliam...,I was struck by the abundance of knowledge whe...,I was struck by the lack of knowledge when I j...
116,116,"Mr President, let me express my thanks for the...","Mr President, I would like to extend a warm we...","Madam President, I would like to extend a warm..."
117,117,We can replace 16 out of 626 Members of the Eu...,"As voters we cannot decide on a new law, but w...","As voters decide on a new law, we can replace ..."


In [10]:
data_df_se

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [11]:
# Replace "'" with '//"'
data_df_en['source'] = data_df_en['source'].str.replace("'", "\\'")

In [12]:
data_df_se['source'] = data_df_se['source'].str.replace("'", "\\'")

In [13]:
display(data_df_se)

,id,source,hyp1,hyp2
0,0,Kvinnor kommer att möta högre bilförsäkringspr...,Det betyder att kvinnor kan förvänta sig att b...,Kvinnor kommer att få högre premier för bilför...
1,1,"Rörelseintäkterna var 1,45 miljarder dollar, e...","Intäkterna från rörelsen var 1,45 miljarder do...","Intäkterna från rörelsen var 1,45 miljarder do..."
2,2,"Mandela tillbaka på sjukhus i ""allvarligt men ...",Mandela återvänder inte till sjukhuset med all...,Mandela återvänder till sjukhuset med allvarli...
3,3,Egypten fryser Muslimska brödraskapets tillgån...,Den egyptiska regeringen har fryst tillgångar ...,Egypten fryser tillgångarna för Muslimska bröd...
4,4,"P-pillret, som de kallar ""polypiller"", skulle ...","Medicinen som vanligtvis kallas ""poly pill"" sä...","Medicinen som vanligtvis kallas ""poly pill"" sä..."
...,...,...,...,...
114,114,Texaner som dignar under skyhöga hemförsäkring...,Invånare i Texas som har belastats med orimlig...,Invånare i Texas som har belastats med orimlig...
115,115,Uppdaterad - Två explosioner nära mållinjen i ...,Uppdatering: Nära målet i Boston Marathon intr...,Två explosioner inträffade under loppet av Bos...
116,116,Anthony Weiner halkar ned till fjärde plats i ...,Anthony Weiner halkar till fjärde plats i ny o...,Enligt en ny opinionsundersökning från Quinnip...
117,117,Östtimor förbjuder kampsportsklubbar på grund ...,I Östtimor ska man förbjuda klubbar som utövar...,I Östtimor har man förbjudit klubbar som utöva...


In [12]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.8 MB/s eta 0:00:00


In [13]:
!pip install -U "transformers==4.38.1" --upgrade
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


# Load the model

In [14]:
#load model Gemma

from transformers import AutoTokenizer, pipeline
import torch

model = "google/gemma-7b-it"
model_id = "google/gemma-7b-it"
dtype = torch.bfloat16

# use quantized model
pipeline = pipeline(
    "text-generation",
    model=model,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True}
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Prompt Version 1

In [15]:
def detect_hallu_narjes(source, hyp1, hyp2):
   # Generate the prompt

   answer_format = {"label": ""}

   user_prompt = f'''
   <start_of_turn>user
   You are a researcher investigating a new phenomenon. You have gathered data ({source}) and formulated two competing hypotheses (hyp1: {hyp1}, and hyp2: {hyp2}) to explain it.
   Identify the hypothesis that contradicts the information provided in the given source.
   Provide the result in the following format: {answer_format}.

   <end_of_turn>

   Result:
   <start_of_turn>model
   '''

   messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

   prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

   terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

   outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
   print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
   hallu_label = outputs[0]["generated_text"][len(prompt):]


   return hallu_label

In [ ]:
def detect_hallu(source, hyp1, hyp2):
   # Generate the prompt

   answer_format = {"label": ""}

   user_prompt = f'''
   <start_of_turn>user
   Given a "src" and two hypotheses "hyp1" and "hyp2" your task is to detect which of the two hypotheses ("label") is not supported by the source.
   Provide the result in the following format: {answer_format}.

   Src: {source}
   hyp1 : {hyp1}
   hyp2 : {hyp2}

   <end_of_turn>

   Result:
   <start_of_turn>model
   '''

   messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

   prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

   terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

   outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
   print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
   hallu_label = outputs[0]["generated_text"][len(prompt):]


   return hallu_label

In [ ]:
def detect_hallu_se(source, hyp1, hyp2):
   # Generate the prompt

   answer_format = {"label": ""}

   user_prompt = f'''
   <start_of_turn>user
   Givet en ”src” och två hypoteser ”hyp1” och ”hyp2” är din uppgift att upptäcka vilken av de två hypoteserna (”label”) som inte stöds av källan.
   Ge resultatet i följande format: {answer_format}.

   Src: {source}
   hyp1 : {hyp1}
   hyp2 : {hyp2}

   <end_of_turn>

   Resultat:
   <start_of_turn>model
   '''

   messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

   prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

   terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

   outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
   print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
   hallu_label = outputs[0]["generated_text"][len(prompt):]


   return hallu_label

In [ ]:
def detect_hallu_2(source, hyp1, hyp2):

  # Generate the prompt

  answer_format = {"label": ""}
  user_prompt =  f'''
      <start_of_turn>user
      As an AI model, your task is to determine which of the two hypotheses, "hyp1" and "hyp2," is not supported by the given source, labeled as either "hyp1" or "hyp2."

      Now, with the provided "src" and two hypotheses "hyp1" and "hyp2," identify the hypothesis that is not supported by the source.

      Src: {source}
      hyp1: {hyp1}
      hyp2: {hyp2}

      DO NOT GENERATE CODE AND ONLY ANSWER WITH THE GIVEN FORMAT USING THE FOLLOWING FORMAT: {answer_format}.



      Example 1:
      Src: The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden.
      hyp1: In the majority of Sweden's 280 municipalities, the population has gone up.
      hyp2: In the majority of Sweden's 280 municipalities, the population has gone down.

      Result:
      '{"label": "hyp1"}'

      Explanation: Hyp1 suggests that the population has increased in most municipalities, which contradicts the information provided in the source. The source clearly states that the population has declined in some municipalities, not increased.


      <start_of_turn>model
      '''

  messages = [
      #{"role": "user", "content": "Imagine you are a classifier"},
      {"role": "user", "content": user_prompt} ]

  prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

  terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

  outputs = pipeline(
        prompt,
        max_new_tokens=150,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

  print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
  hallu_label = outputs[0]["generated_text"][len(prompt):]


  return hallu_label

In [16]:
import json
import re

def extract_json_data(result_text):

    print(result_text)
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    print("result_text:",result_text)
    return result_text

In [ ]:
source = "My mom's food is better than my sister's."
hyp1= "My mom's food is amazing"
hyp2="My sister's food is better than my mom's"
result = detect_hallu_2(source,hyp1,hyp2)
print()
print(result)
result = extract_json_data(result)
print()

print("result:",result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp1'}

The source text describes the decline of the population in Sweden's municipalities, stating that it has decreased in most municipalities. Hyp1 contradicts this information, proposing that the population has increased in most municipalities. Hence, it is detected as not supporting the source.

{'label': 'hyp1'}

The source text describes the decline of the population in Sweden's municipalities, stating that it has decreased in most municipalities. Hyp1 contradicts this information, proposing that the population has increased in most municipalities. Hence, it is detected as not supporting the source.
{'label': 'hyp1'}

The source text describes the decline of the population in Sweden's municipalities, stating that it has decreased in most municipalities. Hyp1 contradicts this information, proposing that the population has increased in most municipalities. Hence, it is detected as not supporting the source.
result_text: {'label': 'hyp1'}

result: {'label': 'h

## Generate dataset

In [17]:
import os
import numpy as np
import pandas as pd

#save results
def save_results(detect_hallu, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)

    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']
            hyp1 = row["hyp1"]
            hyp2 = row["hyp2"]

            try:
                # Generate translation
                result = detect_hallu(source, hyp1, hyp2)

                print(result)
                extracted_hallu = extract_json_data(result)

                hallu_label = extracted_hallu["label"]


                # Update DataFrame with new values
                chunk.at[index, "prediction"] = hallu_label
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)

        # Load CSV file
        df = pd.read_csv(csv_filename)
        #display(df)
    return skipped_rows

# Evaluate prompts on the trial dataset

In [20]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_vnarjes_trial.csv"


save_results(detect_hallu_narjes, trial_df_en, csv_filename)

Gen text:  **Result:**

{'label': 'Hyp1'}

Hyp1 contradicts the information provided in the source, which states that the population has declined in 210 of the 280 municipalities in Sweden.
**Result:**

{'label': 'Hyp1'}

Hyp1 contradicts the information provided in the source, which states that the population has declined in 210 of the 280 municipalities in Sweden.
**Result:**

{'label': 'Hyp1'}

Hyp1 contradicts the information provided in the source, which states that the population has declined in 210 of the 280 municipalities in Sweden.
result_text: {'label': 'Hyp1'}
Gen text:  **Result:**
{'label': 'hyp1'}

The source text states that the draft agenda has been distributed, therefore hyp1 contradicts the information provided in the source text.
**Result:**
{'label': 'hyp1'}

The source text states that the draft agenda has been distributed, therefore hyp1 contradicts the information provided in the source text.
**Result:**
{'label': 'hyp1'}

The source text states that the draft a

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  **Result:**
{'label': 'hyp1'}

The text provided describes a situation where the author believes that listening to everyone's views is necessary to be an ineffective President. Therefore, the hypothesis that contradicts the information provided in the given source is hyp1.
**Result:**
{'label': 'hyp1'}

The text provided describes a situation where the author believes that listening to everyone's views is necessary to be an ineffective President. Therefore, the hypothesis that contradicts the information provided in the given source is hyp1.
**Result:**
{'label': 'hyp1'}

The text provided describes a situation where the author believes that listening to everyone's views is necessary to be an ineffective President. Therefore, the hypothesis that contradicts the information provided in the given source is hyp1.
result_text: {'label': 'hyp1'}
Gen text:  **Result:**
{'label': 'Hyp2'}

The text states that Agenda 2000 does not include a chapter on renewable energy, which contrad

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Result:
The information provided states that the EU wants the information to enter into force within three years from 1 July 1998. Therefore, hyp1 is the correct hypothesis that contradicts the information provided.
{'label': 'hyp2'}

Result:
The information provided states that the EU wants the information to enter into force within three years from 1 July 1998. Therefore, hyp1 is the correct hypothesis that contradicts the information provided.
{'label': 'hyp2'}

Result:
The information provided states that the EU wants the information to enter into force within three years from 1 July 1998. Therefore, hyp1 is the correct hypothesis that contradicts the information provided.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'label': 'hyp2'}

The text describes a situation where the speaker decided to leave their speech aside due to the similarity of points made by previous speakers. The text explicitly states that the speaker left their speech aside because many of the points they were going to make had already been made by previous speakers. Therefore, the hypothesis that contradicts the information provided in the given source is hyp2.
Result: {'label': 'hyp2'}

The text describes a situation where the speaker decided to leave their speech aside due to the similarity of points made by previous speakers. The text explicitly states that the speaker left their speech aside because many of the points they were going to make had already been made by previous speakers. Therefore, the hypothesis that contradicts the information provided in the given source is hyp2.
Result: {'label': 'hyp2'}

The text describes a situation where the speaker decided to leave their speech aside due to the simi

[15]

In [22]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_vnarjes_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/classification_report_gemma_en_vnarjes_trial.csv", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1,hyp2
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2,hyp2
5,5,This state of affairs has not changed in more ...,addition,There has been no change in the status quo in ...,The state of affairs is1-65561-65561-65561-655...,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of ou...",gender,One of the motions for a resolution was drafte...,One of the motions for a resolution was drafte...,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes i...,natural,The French version excludes the expression'pol...,The French version excludes the expression 'po...,hyp1,hyp2
8,8,The European Commission proposes that this inf...,number,The EU wants this information to enter into fo...,The EU wants this information to enter into fo...,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I hav...",pronoun,Many of the points they were going to make in ...,Many of the points I was going to make in my s...,hyp1,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.44      0.62         9
        hyp2       0.58      1.00      0.74         7

    accuracy                           0.69        16
   macro avg       0.79      0.72      0.68        16
weighted avg       0.82      0.69      0.67        16



326

In [28]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_se_vnarjes_trial.csv"


save_results(detect_hallu_narjes, trial_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

The information provided in the source states that the intäkter från programvarulicenser minskade med 21 procent till 107,6 miljoner dollar. Therefore, the hypothesis that contradicts this information is Hyp2.
{'label': 'Hyp2'}

The information provided in the source states that the intäkter från programvarulicenser minskade med 21 procent till 107,6 miljoner dollar. Therefore, the hypothesis that contradicts this information is Hyp2.
{'label': 'Hyp2'}

The information provided in the source states that the intäkter från programvarulicenser minskade med 21 procent till 107,6 miljoner dollar. Therefore, the hypothesis that contradicts this information is Hyp2.
result_text: {'label': 'Hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

Result:
The information provided in the source describes a collaboration between the University of Hong Kong and various kinesian universities, companies and government sectors to promote advanced technological research, commercialization and technology transfer. Therefore, the hypothesis that contradicts this information is Hyp2.
{'label': 'Hyp2'}

Result:
The information provided in the source describes a collaboration between the University of Hong Kong and various kinesian universities, companies and government sectors to promote advanced technological research, commercialization and technology transfer. Therefore, the hypothesis that contradicts this information is Hyp2.
{'label': 'Hyp2'}

Result:
The information provided in the source describes a collaboration between the University of Hong Kong and various kinesian universities, companies and government sectors to promote advanced technological research, commercialization and technology transfer. Th

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

The text states that AstraZeneca has agreed to transfer 4,1 billion dollars to Bristol-Myers Squibb to acquire their diabetes alliance. Therefore, the hypothesis that contradicts this information is hyp2.
{'label': 'Hyp2'}

The text states that AstraZeneca has agreed to transfer 4,1 billion dollars to Bristol-Myers Squibb to acquire their diabetes alliance. Therefore, the hypothesis that contradicts this information is hyp2.
{'label': 'Hyp2'}

The text states that AstraZeneca has agreed to transfer 4,1 billion dollars to Bristol-Myers Squibb to acquire their diabetes alliance. Therefore, the hypothesis that contradicts this information is hyp2.
result_text: {'label': 'Hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

Result:
The information provided in the source indicates that Nikos Michaloliakos is suspected of organizing violent protesters, therefore hyp2 contradicts the information provided in the source.
{'label': 'Hyp2'}

Result:
The information provided in the source indicates that Nikos Michaloliakos is suspected of organizing violent protesters, therefore hyp2 contradicts the information provided in the source.
{'label': 'Hyp2'}

Result:
The information provided in the source indicates that Nikos Michaloliakos is suspected of organizing violent protesters, therefore hyp2 contradicts the information provided in the source.
result_text: {'label': 'Hyp2'}
Gen text:  Result: {'label': 'Hyp2'}

The text states that the information provided is about Israels Peres uppmanar parterna att återgå till fredssamtalen. Therefore, the hypothesis that contradicts the information provided is hyp2.
Result: {'label': 'Hyp2'}

The text states that the information provided is abou

[]

In [29]:
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_se_vnarjes_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/classification_report_gemma_se_vnarjes_trial.csv", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2,hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1,hyp2
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2,hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1,hyp1
5,5,Spaniens prinsessa vittnar i historisk bedräge...,gender,Spanska prinsessan blir en del av en utredning...,Spanska prinsen blir en del av en utredning om...,hyp2,hyp2
6,6,Mannen använder en slägga för att bryta betong...,antonym,Mannen använder en slägga för att krossa beton...,Mannen använder en slägga för att krossa beton...,hyp2,hyp2
7,7,Nordkorea carnar utlänningar i söder och ger r...,negation,Nordkorea varnar inte utländska medborgare i s...,Nordkorea varnar utländska medborgare i södra ...,hyp1,hyp1
8,8,Beväpnad man bland 7 döda efter lägenhetsskjut...,addition,En beväpnad man var bland de sju döda efter en...,En man med ett skjutvapen har dödat minst sju ...,hyp2,hyp2
9,9,Irans kärnvapenförhandlingar går in på tredje ...,number,Diskussioner om irans kärnvapenprogram har nåt...,Diskussioner om irans kärnvapenprogram har nåt...,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.22      0.36         9
        hyp2       0.61      1.00      0.76        11

    accuracy                           0.65        20
   macro avg       0.81      0.61      0.56        20
weighted avg       0.79      0.65      0.58        20



326

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_v1_trial.csv"


save_results(detect_hallu, trial_df_en, csv_filename)

Gen text:  ```python
result = {'label': 'hyp1'}

src = "The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden."

hyp1 = "In the majority of Sweden's 280 municipalities, the population has gone up."
hyp2 = "In the majority of Sweden's 280 municipalities, the population has gone down."

if 'declined' in src or 'down' in src:
    result['label'] = hyp2
elif 'up' in src:
    result['label'] = hyp1

print(result)
```

**Output:**

```
{'label':
```python
result = {'label': 'hyp1'}

src = "The population has declined in some 210 of the 280 municipalities in Sweden, mainly in inland central and northern Sweden."

hyp1 = "In the majority of Sweden's 280 municipalities, the population has gone up."
hyp2 = "In the majority of Sweden's 280 municipalities, the population has gone down."

if 'declined' in src or 'down' in src:
    result['label'] = hyp2
elif 'up' in src:
    result['label'] = hyp1

print(result)
```

**Output:**

`

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp1'}

src = "The draft agenda as drawn up by the Conference of Presidents pursuant to Rule 95 of the Rules of Procedure has been distributed."

hyp1 = "The Conference of Presidents hasn't distributed the draft agenda."
hyp2 = "The Conference of Presidents has distributed the draft agenda."

if "has been distributed" in src:
    result['label'] = 'hyp2'

print(result)
```

**Output:**
```
{'label': 'hyp2'}
```

**Explanation:**

The source text states that the draft agenda has been distributed. Therefore, the hypothesis 'hyp2' is supported by the source, while
```python
result = {'label': 'hyp1'}

src = "The draft agenda as drawn up by the Conference of Presidents pursuant to Rule 95 of the Rules of Procedure has been distributed."

hyp1 = "The Conference of Presidents hasn't distributed the draft agenda."
hyp2 = "The Conference of Presidents has distributed the draft agenda."

if "has been distributed" in src:
    result['label'] = 'hyp2'

pri

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "The European Commission proposes that this information should enter into force within a period of three years from 1 July 1998."

hyp1 = "The EU wants this information to enter into force in three years."
hyp2 = "The EU wants this information to enter into force in thirty years."

if 'three years' in src and 'thirty years' not in src:
    result['label'] = hyp2

print(result)
```

**Result:**

```
{'label': 'hyp2'}
```

**Explanation:**

* The source text mentions "three years" but not "thirty years".
*
```python
result = {'label': 'hyp2'}

src = "The European Commission proposes that this information should enter into force within a period of three years from 1 July 1998."

hyp1 = "The EU wants this information to enter into force in three years."
hyp2 = "The EU wants this information to enter into force in thirty years."

if 'three years' in src and 'thirty years' not in src:
    result['label'] = hyp2

print(result)
```

**Resu

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "Mr President, I did prepare a speech but I have left it aside because many of my points have already been excellently made by previous speakers."

hyp1 = "Many of the points they were going to make in their speech were already made by previous speakers, so they left it aside."

hyp2 = "Many of the points I was going to make in my speech were already made by previous speakers, so I left it aside."

if hyp1 in src:
    result['label'] = 'hyp1'

print(result)
```

**Output:**

```
{'label': 'hyp2'}
```

**Explanation:**


```python
result = {'label': 'hyp2'}

src = "Mr President, I did prepare a speech but I have left it aside because many of my points have already been excellently made by previous speakers."

hyp1 = "Many of the points they were going to make in their speech were already made by previous speakers, so they left it aside."

hyp2 = "Many of the points I was going to make in my speech were already made by previous speak

[]

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/classification_report_gemma_en_v1_trial.txt", "w") as text_file:
    text_file.write(report)


,id,source,type,hyp1,hyp2,label,prediction
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2,hyp2
5,5,This state of affairs has not changed in more ...,addition,There has been no change in the status quo in ...,The state of affairs is1-65561-65561-65561-655...,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of ou...",gender,One of the motions for a resolution was drafte...,One of the motions for a resolution was drafte...,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes i...,natural,The French version excludes the expression'pol...,The French version excludes the expression 'po...,hyp1,hyp2
8,8,The European Commission proposes that this inf...,number,The EU wants this information to enter into fo...,The EU wants this information to enter into fo...,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I hav...",pronoun,Many of the points they were going to make in ...,Many of the points I was going to make in my s...,hyp1,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.78      0.88         9
        hyp2       0.78      1.00      0.88         7

    accuracy                           0.88        16
   macro avg       0.89      0.89      0.88        16
weighted avg       0.90      0.88      0.88        16



326

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_v2_trial.csv"


save_results(detect_hallu_2, trial_df_en, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp1'}

The source text describes a decline in the population in 210 municipalities in Sweden. Therefore, the hypothesis that is not supported by the source is hyp1.
{'label': 'hyp1'}

The source text describes a decline in the population in 210 municipalities in Sweden. Therefore, the hypothesis that is not supported by the source is hyp1.
{'label': 'hyp1'}

The source text describes a decline in the population in 210 municipalities in Sweden. Therefore, the hypothesis that is not supported by the source is hyp1.
result_text: {'label': 'hyp1'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'label': 'hyp1'}

The source text states that the draft agenda has been distributed, therefore, hyp1 is not supported by the source.
Result: {'label': 'hyp1'}

The source text states that the draft agenda has been distributed, therefore, hyp1 is not supported by the source.
Result: {'label': 'hyp1'}

The source text states that the draft agenda has been distributed, therefore, hyp1 is not supported by the source.
result_text: {'label': 'hyp1'}
Gen text:  {'label': 'hyp1'}

The source text does not support hyp1. The text suggests that the President needs to listen to everyone's views in order to be effective, not ineffective.
{'label': 'hyp1'}

The source text does not support hyp1. The text suggests that the President needs to listen to everyone's views in order to be effective, not ineffective.
{'label': 'hyp1'}

The source text does not support hyp1. The text suggests that the President needs to listen to everyone's views in order to be effective, not ineffective.

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

The source text states that the information should enter into force within three years from 1 July 1998. Therefore, the hypothesis that is not supported by the source is hyp2, which proposes that the information should enter into force in thirty years.
{'label': 'hyp2'}

The source text states that the information should enter into force within three years from 1 July 1998. Therefore, the hypothesis that is not supported by the source is hyp2, which proposes that the information should enter into force in thirty years.
{'label': 'hyp2'}

The source text states that the information should enter into force within three years from 1 July 1998. Therefore, the hypothesis that is not supported by the source is hyp2, which proposes that the information should enter into force in thirty years.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

The source text clearly states that "many of the points I was going to make in my speech were already made by previous speakers," therefore, hyp2 is not supported by the source.
{'label': 'hyp2'}

The source text clearly states that "many of the points I was going to make in my speech were already made by previous speakers," therefore, hyp2 is not supported by the source.
{'label': 'hyp2'}

The source text clearly states that "many of the points I was going to make in my speech were already made by previous speakers," therefore, hyp2 is not supported by the source.
result_text: {'label': 'hyp2'}
Gen text:  {'label': 'hyp1'}

The source states that "We have done so: on 5 February we published an extremely detailed press release dealing with the questions you have raised." Therefore, the hypothesis that is not supported by the source is hyp1.
{'label': 'hyp1'}

The source states that "We have done so: on 5 February we published an extremely detailed press re

[]

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_en_v2_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/classification_report_gemma_en_v2_trial.txt", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1,hyp2
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2,hyp2
5,5,This state of affairs has not changed in more ...,addition,There has been no change in the status quo in ...,The state of affairs is1-65561-65561-65561-655...,hyp2,hyp2
6,6,"Madam President, I am speaking on behalf of ou...",gender,One of the motions for a resolution was drafte...,One of the motions for a resolution was drafte...,hyp2,hyp2
7,7,Amendment No 1 in the French version deletes i...,natural,The French version excludes the expression'pol...,The French version excludes the expression 'po...,hyp1,hyp2
8,8,The European Commission proposes that this inf...,number,The EU wants this information to enter into fo...,The EU wants this information to enter into fo...,hyp2,hyp2
9,9,"Mr President, I did prepare a speech but I hav...",pronoun,Many of the points they were going to make in ...,Many of the points I was going to make in my s...,hyp1,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.67      0.80         9
        hyp2       0.70      1.00      0.82         7

    accuracy                           0.81        16
   macro avg       0.85      0.83      0.81        16
weighted avg       0.87      0.81      0.81        16



326

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_se_v1_trial.csv"


save_results(detect_hallu_se, trial_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
{'label': 'hyp2'}

Src: Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
hyp1 : Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2 : Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,
```python
{'label': 'hyp2'}

Src: Men intäkterna från mjukvarulicenser, ett mått som finansanalytiker följer noga, minskade med 21 procent till 107,6 miljoner dollar.
hyp1 : Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 21 procent till ett belopp av 107,6 miljoner dollar.
hyp2 : Intäkter från programvarulicenser, en metrik som noggrant övervakas av finansiella analytiker, minskade med 42 procent till ett belopp av 107,
```python
{'label':

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Källan stöds av hyp1, men inte hyp2.
{'label': 'hyp2'}

Källan stöds av hyp1, men inte hyp2.
{'label': 'hyp2'}

Källan stöds av hyp1, men inte hyp2.
result_text: {'label': 'hyp2'}
Gen text:  ```python
{'label': 'hyp2'}

Src: Län med befolkningsminskning kommer att vara Vermillion, Posey och Madison.

hyp1 : Vermillion, Posey och Madison är län som kommer att uppleva minskande befolkning.
hyp2 : Vermillion, Posey och Marion är län som kommer att uppleva minskande befolkning.

Resultat:
{'label': 'hyp2'}
```

**Resultat:**

Hypotesen "hyp2" är inte stödd av källan. Källan säger att län med befolkningsminskning kommer att vara Vermillion, Posey och Madison. Den inte med Marion.
```python
{'label': 'hyp2'}

Src: Län med befolkningsminskning kommer att vara Vermillion, Posey och Madison.

hyp1 : Vermillion, Posey och Madison är län som kommer att uppleva minskande befolkning.
hyp2 : Vermillion, Posey och Marion är län som kommer att uppleva minskande befolkning

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Källan indikerar att AstraZeneca har komma överens om att överföra 4,1 miljarder dollar till Bristol-Myers Squibb för att köpa ut företaget ur diabetesalliansen, inte för att köpa ut diabetesalliansen.
{'label': 'hyp2'}

Källan indikerar att AstraZeneca har komma överens om att överföra 4,1 miljarder dollar till Bristol-Myers Squibb för att köpa ut företaget ur diabetesalliansen, inte för att köpa ut diabetesalliansen.
{'label': 'hyp2'}

Källan indikerar att AstraZeneca har komma överens om att överföra 4,1 miljarder dollar till Bristol-Myers Squibb för att köpa ut företaget ur diabetesalliansen, inte för att köpa ut diabetesalliansen.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Källan stöds inte hyp1, men den stöds hyp2.
{'label': 'hyp2'}

Källan stöds inte hyp1, men den stöds hyp2.
{'label': 'hyp2'}

Källan stöds inte hyp1, men den stöds hyp2.
result_text: {'label': 'hyp2'}
Gen text:  {'label': 'hyp2'}

Källan säger att Israels president Shimon Peres uppmanade på måndagen världssamfundet att återuppta fredsförhandlingarna med palestinierna och betonade vikten av en tvåstatslösning.

Hyp1 säger att Peres i Israel ger en uppmaning att återgå till fredssamtalen.

Källan stöds inte hyp1, utan stöds hyp2.
{'label': 'hyp2'}

Källan säger att Israels president Shimon Peres uppmanade på måndagen världssamfundet att återuppta fredsförhandlingarna med palestinierna och betonade vikten av en tvåstatslösning.

Hyp1 säger att Peres i Israel ger en uppmaning att återgå till fredssamtalen.

Källan stöds inte hyp1, utan stöds hyp2.
{'label': 'hyp2'}

Källan säger att Israels president Shimon Peres uppmanade på måndagen världssamfundet att återu

[]

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/results_gemma_se_v1_trial.csv")

display(df)

# Erstelle den Klassifikationsbericht
report = classification_report(df['label'], df['prediction'])

print(report)

# Speichere den Klassifikationsbericht in eine Textdatei
with open("/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/trial/classification_report_gemma_se_v1_trial.txt", "w") as text_file:
    text_file.write(report)

,id,source,type,hyp1,hyp2,label,prediction
0,0,"Men intäkterna från mjukvarulicenser, ett mått...",number,"Intäkter från programvarulicenser, en metrik s...","Intäkter från programvarulicenser, en metrik s...",hyp2,hyp2
1,1,Hongkong-universitet samarbetar med universite...,natural,University of Hong Kong samarbetar med olika k...,University of Hong Kong samarbetar med olika k...,hyp1,hyp2
2,2,Län med befolkningsminskning kommer att vara V...,named entity,"Vermillion, Posey och Madison är län som komme...","Vermillion, Posey och Marion är län som kommer...",hyp2,hyp2
3,3,Google presenterar en prototyp för en självkör...,addition,En prototyp för en självkörande bil presentera...,Enligt ett blogginlägg från Google har företag...,hyp2,hyp2
4,4,Lagförslaget säger att en kvinna som genomgår ...,negation,Förslaget stadgar att en kvinna som genomgår e...,Förslaget stadgar att en kvinna som genomgår e...,hyp1,hyp1
5,5,Spaniens prinsessa vittnar i historisk bedräge...,gender,Spanska prinsessan blir en del av en utredning...,Spanska prinsen blir en del av en utredning om...,hyp2,hyp2
6,6,Mannen använder en slägga för att bryta betong...,antonym,Mannen använder en slägga för att krossa beton...,Mannen använder en slägga för att krossa beton...,hyp2,hyp2
7,7,Nordkorea carnar utlänningar i söder och ger r...,negation,Nordkorea varnar inte utländska medborgare i s...,Nordkorea varnar utländska medborgare i södra ...,hyp1,hyp2
8,8,Beväpnad man bland 7 döda efter lägenhetsskjut...,addition,En beväpnad man var bland de sju döda efter en...,En man med ett skjutvapen har dödat minst sju ...,hyp2,hyp2
9,9,Irans kärnvapenförhandlingar går in på tredje ...,number,Diskussioner om irans kärnvapenprogram har nåt...,Diskussioner om irans kärnvapenprogram har nåt...,hyp2,hyp2


              precision    recall  f1-score   support

        hyp1       1.00      0.33      0.50         9
        hyp2       0.65      1.00      0.79        11

    accuracy                           0.70        20
   macro avg       0.82      0.67      0.64        20
weighted avg       0.81      0.70      0.66        20



326

# Generate predictions on the test data

## English

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_en_v1.csv"

save_results(detect_hallu, data_df_en, csv_filename)

Gen text:  ```python
result = {'label': 'hyp2'}

src = "It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe."

hyp1 = "You can support and encourage an exchange of experiences to raise the level of competence in Europe."
hyp2 = "We can support and encourage an exchange of experiences to raise the level of competence in Europe."

if hyp1 in src and hyp2 not in src:
    result['label'] = hyp2

print(result)
```

**Output:**

```
{'label': 'hyp2'}
```

**Explanation:**

* The source text mentions "support and encourage an exchange of experiences
```python
result = {'label': 'hyp2'}

src = "It has enabled us to support and encourage an exchange of experiences and to pursue activities to raise the level of competence throughout Europe."

hyp1 = "You can support and encourage an exchange of experiences to raise the level of competence in Europe."
hyp2 = "We can support and encourage an exchange of e

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "As a result, I cannot promise the chairman of the Committee on Budgets that there will be 100 % funding for the financial programme."

hyp1 = "I can't promise that there will be 100 percent funding for the financial program."

hyp2 = "Because of this, it's not possible for me to promise the chairman of the Committee on Budgets that there would be 210 % funding for the financial programme."

if hyp2 not in src:
    result['label'] = hyp2

print(result)
```

**Output:**

```
{'label': 'hyp2'}
```
```python
result = {'label': 'hyp2'}

src = "As a result, I cannot promise the chairman of the Committee on Budgets that there will be 100 % funding for the financial programme."

hyp1 = "I can't promise that there will be 100 percent funding for the financial program."

hyp2 = "Because of this, it's not possible for me to promise the chairman of the Committee on Budgets that there would be 210 % funding for the financial programme."

if hy

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "We voted for Amendments Nos 22 to 25, even though they do not take adequate measures to prevent these problems from happening."

hyp1 = "Amendments Nos 22 to 25 didn't take enough measures to prevent problems from happening, so we voted for them."

hyp2 = "Amendments Nos 22 to 25 didn't take enough measures to prevent problems from happening, yet we voted for them."

if hyp1 not in src or hyp2 in src:
    result['label'] = hyp2

print(result)
```

**Output:**

```
{'label': '
```python
result = {'label': 'hyp2'}

src = "We voted for Amendments Nos 22 to 25, even though they do not take adequate measures to prevent these problems from happening."

hyp1 = "Amendments Nos 22 to 25 didn't take enough measures to prevent problems from happening, so we voted for them."

hyp2 = "Amendments Nos 22 to 25 didn't take enough measures to prevent problems from happening, yet we voted for them."

if hyp1 not in src or hyp2 in src:
    result['l

...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
Result = {'label': 'hyp1'}

Src = "An important factor here is that the pay and other working conditions of those employed in navigation should be in line with those of other occupations."

hyp1 = "The pay and working conditions of those employed in navigation should be better than those of other jobs."
hyp2 = "The pay and working conditions of those employed in navigation should match those of other jobs."

If the source text supports hyp2, then the result should be:

```python
Result = {'label': 'hyp2'}
```

In this case, the source text does not support hyp1, therefore the result is:

```python
Result = {'label': 'hyp1
```python
Result = {'label': 'hyp1'}

Src = "An important factor here is that the pay and other working conditions of those employed in navigation should be in line with those of other occupations."

hyp1 = "The pay and working conditions of those employed in navigation should be better than those of other jobs."
hyp2 = "The pay and working condit

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
result = {'label': 'hyp2'}

src = "Indeed, without the chairmanship skills of the Deputy Prime Minister, John Prescott, the Conference would have produced more hot air than it prevented!"

hyp1 = "The Conference wouldn't have produced as much hot air if it weren't for the chairmanship skills of the deputy prime minister."

hyp2 = "The Conference would have made more hot air than it prevented if it weren't for the chairmanship skills of the deputy prime minister!"

if hyp1 not in src:
    result['label'] = hyp1

print(result)
```

**Output:**

```
{'label': 'hyp2'}
```

**
```python
result = {'label': 'hyp2'}

src = "Indeed, without the chairmanship skills of the Deputy Prime Minister, John Prescott, the Conference would have produced more hot air than it prevented!"

hyp1 = "The Conference wouldn't have produced as much hot air if it weren't for the chairmanship skills of the deputy prime minister."

hyp2 = "The Conference would have made more hot air than it preve

[]

In [ ]:
# save a csv file in the task format

import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_en_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_en_v1_detection.csv', index=False)

In [23]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_en_vnarjes.csv"

save_results(detect_hallu_narjes, data_df_en, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'label': hyp2}

The text describes a phenomenon and two hypotheses to explain it. The text states that hyp1 is supported by the information provided, therefore hyp2 contradicts the information provided.
Result: {'label': hyp2}

The text describes a phenomenon and two hypotheses to explain it. The text states that hyp1 is supported by the information provided, therefore hyp2 contradicts the information provided.
Result: {'label': hyp2}

The text describes a phenomenon and two hypotheses to explain it. The text states that hyp1 is supported by the information provided, therefore hyp2 contradicts the information provided.
Skipping row 0 due to error in data extraction.
Gen text:  **Result:**

{'label': 'hyp1'}

The information provided in the source indicates an increase in premiums for all varieties of leaf tobacco for the 1999, 2000 and 2001 harvests. Therefore, the hypothesis that contradicts this information is hyp1.
**Result:**

{'label': 'hyp1'}

The information 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  **Result:**

{'label': 'Hyp2'}

The information provided in the source indicates that the hypothesis that contradicts the information is hyp1.
**Result:**

{'label': 'Hyp2'}

The information provided in the source indicates that the hypothesis that contradicts the information is hyp1.
**Result:**

{'label': 'Hyp2'}

The information provided in the source indicates that the hypothesis that contradicts the information is hyp1.
result_text: {'label': 'Hyp2'}
Gen text:  Result: {'label': 'hyp2'}

The text states that none of the 34 amendments were adopted and the arguments against the administrative burden were not heeded. Therefore, the hypothesis that contradicts this information is hyp2.
Result: {'label': 'hyp2'}

The text states that none of the 34 amendments were adopted and the arguments against the administrative burden were not heeded. Therefore, the hypothesis that contradicts this information is hyp2.
Result: {'label': 'hyp2'}

The text states that none of the 34 amend

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  **Result:**

{'label': 'hyp2'}

The text states that the pay and working conditions of those employed in navigation should match those of other jobs, which contradicts the hypothesis 'hyp1'.
**Result:**

{'label': 'hyp2'}

The text states that the pay and working conditions of those employed in navigation should match those of other jobs, which contradicts the hypothesis 'hyp1'.
**Result:**

{'label': 'hyp2'}

The text states that the pay and working conditions of those employed in navigation should match those of other jobs, which contradicts the hypothesis 'hyp1'.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'hyp2': ''}.

The text states that the conference produced more hot air than it prevented, therefore hyp2 contradicts the information provided.
Result: {'hyp2': ''}.

The text states that the conference produced more hot air than it prevented, therefore hyp2 contradicts the information provided.
Result: {'hyp2': ''}.

The text states that the conference produced more hot air than it prevented, therefore hyp2 contradicts the information provided.
result_text: {'hyp2': ''}
Skipping row 111 due to error in data extraction.
Gen text:  **Result:**

{'label': 'hyp2'}

The information provided in the source suggests that hyp1 is the hypothesis that contradicts the information provided.
**Result:**

{'label': 'hyp2'}

The information provided in the source suggests that hyp1 is the hypothesis that contradicts the information provided.
**Result:**

{'label': 'hyp2'}

The information provided in the source suggests that hyp1 is the hypothesis that contradicts the information 

[0,
 7,
 9,
 11,
 21,
 23,
 31,
 32,
 44,
 46,
 47,
 49,
 53,
 54,
 62,
 63,
 67,
 70,
 76,
 80,
 83,
 84,
 90,
 95,
 97,
 102,
 111,
 117]

In [27]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_en_vnarjes.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_en_vnarjes.csv', index=False)

In [30]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_se_vnarjes.csv"

save_results(detect_hallu_narjes, data_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'label': 'hyp2'}

The information provided in the source states that Kvinnor kommer att få högre premier för bilförsäkring. This information contradicts the hypothesis hyp2.
Result: {'label': 'hyp2'}

The information provided in the source states that Kvinnor kommer att få högre premier för bilförsäkring. This information contradicts the hypothesis hyp2.
Result: {'label': 'hyp2'}

The information provided in the source states that Kvinnor kommer att få högre premier för bilförsäkring. This information contradicts the hypothesis hyp2.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

The text describes a situation where the income from movement was 1,45 billion dollars, which is an increase compared to the previous year's result of 1,38 billion dollars. Therefore, the hypothesis that contradicts the information provided in the given source is hyp2.
{'label': 'Hyp2'}

The text describes a situation where the income from movement was 1,45 billion dollars, which is an increase compared to the previous year's result of 1,38 billion dollars. Therefore, the hypothesis that contradicts the information provided in the given source is hyp2.
{'label': 'Hyp2'}

The text describes a situation where the income from movement was 1,45 billion dollars, which is an increase compared to the previous year's result of 1,38 billion dollars. Therefore, the hypothesis that contradicts the information provided in the given source is hyp2.
result_text: {'label': 'Hyp2'}
Gen text:  Result: {'label': hyp1}

The text describes a phenomenon and provides informatio

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  Result: {'label': 'hyp2'}

The text states that the source information indicates that Obama's speech on Syria is scheduled for 13:15 EDT. Therefore, hyp2 contradicts the information provided in the source.
Result: {'label': 'hyp2'}

The text states that the source information indicates that Obama's speech on Syria is scheduled for 13:15 EDT. Therefore, hyp2 contradicts the information provided in the source.
Result: {'label': 'hyp2'}

The text states that the source information indicates that Obama's speech on Syria is scheduled for 13:15 EDT. Therefore, hyp2 contradicts the information provided in the source.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'Hyp2'}

Result:

Hyp2 är den hypotes som motstänger informationen som är tillgänglig i den angivna källan.
{'label': 'Hyp2'}

Result:

Hyp2 är den hypotes som motstänger informationen som är tillgänglig i den angivna källan.
{'label': 'Hyp2'}

Result:

Hyp2 är den hypotes som motstänger informationen som är tillgänglig i den angivna källan.
result_text: {'label': 'Hyp2'}
Gen text:  Result: {'label': 'Hyp2'}

The information provided in the source describes a black cat lying on a mat. The text does not mention a sweet, small kitten. Therefore, the hypothesis that contradicts the information provided in the source is Hyp2.
Result: {'label': 'Hyp2'}

The information provided in the source describes a black cat lying on a mat. The text does not mention a sweet, small kitten. Therefore, the hypothesis that contradicts the information provided in the source is Hyp2.
Result: {'label': 'Hyp2'}

The information provided in the source describes a black cat lying on a mat. T

[2, 47, 65, 97]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_se_vnarjes.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_se_vnarjes.csv', index=False)

In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_en_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_en_v1_detection.csv', index=False)

## Swedish

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/"

save_results(detect_hallu_se, data_df_se, csv_filename)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  ```python
{'label': 'hyp2'}

Src: Kvinnor kommer att möta högre bilförsäkringspremier.

hyp1 : Det betyder att kvinnor kan förvänta sig att betala högre priser för sina fordonstilläggsförsäkringar.
hyp2 : Kvinnor kommer att få högre premier för bilförsäkring.

Resultat:
{'label': 'hyp2'}
```

**Resultat:**

Hypotesen som inte stöds av källan är hyp2. Källan säger att kvinnor kommer att möta högre bilförsäkringspremier, men det säger inte att kvinnor kommer att få högre premier för bilförsäkring.
```python
{'label': 'hyp2'}

Src: Kvinnor kommer att möta högre bilförsäkringspremier.

hyp1 : Det betyder att kvinnor kan förvänta sig att betala högre priser för sina fordonstilläggsförsäkringar.
hyp2 : Kvinnor kommer att få högre premier för bilförsäkring.

Resultat:
{'label': 'hyp2'}
```

**Resultat:**

Hypotesen som inte stöds av källan är hyp2. Källan säger att kvinnor kommer att möta högre bilförsäkringspremier, men det säger inte att kvinnor kommer att få högre premier för bi

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp1'}

Källan indikerar att intäkterna från rörelsen var 1,45 miljarder dollar, vilket är en ökning jämfört med förra årets resultat på 1,38 miljarder dollar. Therefore, hyp1 är inte stödd av källan.
{'label': 'hyp1'}

Källan indikerar att intäkterna från rörelsen var 1,45 miljarder dollar, vilket är en ökning jämfört med förra årets resultat på 1,38 miljarder dollar. Therefore, hyp1 är inte stödd av källan.
{'label': 'hyp1'}

Källan indikerar att intäkterna från rörelsen var 1,45 miljarder dollar, vilket är en ökning jämfört med förra årets resultat på 1,38 miljarder dollar. Therefore, hyp1 är inte stödd av källan.
result_text: {'label': 'hyp1'}
Gen text:  ```python
{'label': 'hyp2'}

Src: Mandela tillbaka på sjukhus i "allvarligt men stabilt" tillstånd.

hyp1 : Mandela återvänder inte till sjukhuset med allvarligt, men stabilt, hälsotillstånd.
hyp2 : Mandela återvänder till sjukhuset med allvarligt, men stabilt, hälsotillstånd.

Resultat:
{'label': 'hyp2'}
```


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  **Resultat:**

{'label': 'hyp2'}

Källan säger att Obama kommer att tala om Syrien från Vitahuset klockan 13:15 EDT. Hyp1 stöds av källan, medan hyp2 inte stöds.
**Resultat:**

{'label': 'hyp2'}

Källan säger att Obama kommer att tala om Syrien från Vitahuset klockan 13:15 EDT. Hyp1 stöds av källan, medan hyp2 inte stöds.
**Resultat:**

{'label': 'hyp2'}

Källan säger att Obama kommer att tala om Syrien från Vitahuset klockan 13:15 EDT. Hyp1 stöds av källan, medan hyp2 inte stöds.
result_text: {'label': 'hyp2'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Gen text:  {'label': 'hyp2'}

Källan säger att Ryssland ratificerade den ändrade versionen av fördraget, men den säger inte att Rysslands regering har godkänt och undertecknat det reviderade fördraget om icke-spridning av kärnvapen (NPT) i New York City, USA, den 26 september 2022.
{'label': 'hyp2'}

Källan säger att Ryssland ratificerade den ändrade versionen av fördraget, men den säger inte att Rysslands regering har godkänt och undertecknat det reviderade fördraget om icke-spridning av kärnvapen (NPT) i New York City, USA, den 26 september 2022.
{'label': 'hyp2'}

Källan säger att Ryssland ratificerade den ändrade versionen av fördraget, men den säger inte att Rysslands regering har godkänt och undertecknat det reviderade fördraget om icke-spridning av kärnvapen (NPT) i New York City, USA, den 26 september 2022.
result_text: {'label': 'hyp2'}
Gen text:  {'label': 'hyp2'}

Källan stöds inte av hyp2. Källan visar en svartvit bild av en katt på en matta, vilket inte är i överensstämmel

[]

In [ ]:
# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/detection_task_gemma/results_gemma_se_v1.csv')

# Select only the columns 'id', 'hyp+', and 'hyp-'
df = df[['id', 'prediction']]
df = df.rename(columns={'prediction': 'label'})

# Save the new DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group2_results/detection_task/final_gemma_se_v1_detection.csv', index=False)

In [ ]:
display(df)

,id,label
0,0,hyp2
1,1,hyp1
2,2,hyp2
3,3,hyp2
4,4,hyp2
...,...,...
114,114,hyp2
115,115,hyp2
116,116,hyp2
117,117,hyp2
